# CITS5504 Data Warehousing Project 2 - Graph Database Design and Query

In [1]:
# Import libraries
import pandas as pd

# Load in data
fifa_df = pd.read_csv('./data/FIFA2014 - all players.csv')

In [2]:
# Inspect the first 5 rows of the data
fifa_df.head()

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False


In [3]:
fifa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 736 entries, 0 to 735
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Player id               736 non-null    int64 
 1   Player                  736 non-null    object
 2   Position                736 non-null    object
 3   Number                  736 non-null    int64 
 4   Club                    736 non-null    object
 5   Club (country)          736 non-null    object
 6   D.O.B                   736 non-null    object
 7   Age                     736 non-null    int64 
 8   Height (cm)             736 non-null    int64 
 9   Country                 736 non-null    object
 10  Caps                    736 non-null    int64 
 11  International goals     736 non-null    int64 
 12  Plays in home country?  736 non-null    bool  
dtypes: bool(1), int64(6), object(6)
memory usage: 69.8+ KB


In [4]:
print(f"Number of instances in the dataset: {len(fifa_df)}")

# Confirm that there are no duplicated players based on Player id
num_unique_players = len(fifa_df["Player id"].unique())
print(f"Number of unique players: {num_unique_players}")

Number of instances in the dataset: 736
Number of unique players: 736


## Create nodes

* Player  
* Club  
* Country

In [5]:
# Create Player node
player_node = fifa_df[["Player id", "Player", "Position", "D.O.B", "Age", "Height (cm)"]]

# Rename columns
player_node.rename(columns={
    "Player id": "player_id", 
    "Player": "name", 
    "Position": "position",
    "D.O.B": "dob", 
    "Age": "age", 
    "Height (cm)": "height"
}, inplace=True)

player_node.to_csv('./data/player_node.csv', index=False)

/tmp/ipykernel_356025/4241665025.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_node.rename(columns={


In [6]:
# Create Club node
club_node = pd.DataFrame(fifa_df["Club"].sort_values().unique(), columns=["Club"])

# Add key column 'club_id'
club_node["club_id"] = range(1, len(club_node) + 1)
# Reorder columns so 'club_id' is first
club_node = club_node[["club_id", "Club"]]

# Merge 'club_id' back into fifa_df
fifa_df = pd.merge(fifa_df, club_node, on="Club", how="left")
# fifa_df[fifa_df["Club"] == "Tigres UANL"] # uncomment to see that merge worked correctly

# Rename columns
club_node.rename(columns={"Club": "club"}, inplace=True)

club_node.to_csv('./data/club_node.csv', index=False)

In [7]:
# Create Country node

# Check which column to use to extract countries - 'Club (country)' or 'Country'
club_ctry = set(fifa_df["Club (country)"].unique())
ctry = set(fifa_df["Country"].unique())
print(f"Number of countries that are unique to the 'Club (country)' column: {len(club_ctry - ctry)}")
print(f"Number of countries that are unique to the 'Country' column: {len(ctry - club_ctry)}")

# Use 'Club (country)' to construct Country node
country_node = pd.DataFrame(fifa_df["Club (country)"].sort_values().unique(), columns=["Country"])
# Add key column 'country_id'
country_node["country_id"] = range(1, len(country_node) + 1)
# Reorder columns so 'country_id' is first
country_node = country_node[["country_id", "Country"]]

# Merge 'country_id' back into fifa_df
fifa_df = pd.merge(fifa_df, country_node, left_on="Club (country)", right_on="Country", how="left")
fifa_df.rename(columns={"country_id": "club_country_id", "Country_x": "Country"}, inplace=True)
fifa_df.drop(columns=["Country_y"], inplace=True) # Drop duplicated column from merge

fifa_df = pd.merge(fifa_df, country_node, on="Country", how="left")
# fifa_df.head() # uncomment to see the merge worked correctly

# Rename column
country_node.rename(columns={"Country": "country"}, inplace=True)

country_node.to_csv('./data/country_node.csv', index=False)

Number of countries that are unique to the 'Club (country)' column: 19
Number of countries that are unique to the 'Country' column: 0


## Create relationships

* (player) PLAYS_FOR (club)  
* (player) FROM (country)  
* (club) LOCATED_IN (country)  

In [8]:
# fifa_df after merges
fifa_df.head()

,Player id,Player,Position,Number,Club,Club (country),D.O.B,Age,Height (cm),Country,Caps,International goals,Plays in home country?,club_id,club_country_id,country_id
0,336722,Alan PULIDO,Forward,11,Tigres UANL,Mexico,08.03.1991,23,176,Mexico,5,4,True,265,31,31
1,368902,Adam TAGGART,Forward,9,Newcastle United Jets FC,Australia,02.06.1993,21,172,Australia,4,3,True,198,3,3
2,362641,Reza GHOOCHANNEJAD,Forward,16,Charlton Athletic FC,England,20.09.1987,26,181,Iran,13,9,False,72,18,25
3,314197,NEYMAR,Forward,10,FC Barcelona,Spain,05.02.1992,22,175,Brazil,48,31,False,102,43,7
4,212306,Didier DROGBA,Forward,11,Galatasaray SK,Turkey,11.03.1978,36,180,Ivory Coast,100,61,False,145,47,28


In [9]:
# Create PLAYS_FOR relationship
rel_plays_for = fifa_df[["Player id", "club_id"]]
# Rename column
rel_plays_for.rename(columns={"Player id": "player_id"}, inplace=True)
rel_plays_for.to_csv('./data/rel_plays_for.csv', index=False)

/tmp/ipykernel_356025/805884145.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_plays_for.rename(columns={"Player id": "player_id"}, inplace=True)


In [10]:
# Create FROM relationship
rel_from = fifa_df[["Player id", "country_id", "Caps", "International goals", "Number"]]
rel_from.rename(columns={
    "Player id": "player_id", 
    "Caps": "caps", 
    "International goals": "intl_goals",
    "Number": "jersey_num", 
}, inplace=True)
rel_from.to_csv('./data/rel_from.csv', index=False)

/tmp/ipykernel_356025/2706328573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_from.rename(columns={


In [11]:
# Create LOCATED_IN relationship
rel_located_in = fifa_df[["club_id", "club_country_id"]].drop_duplicates()

# Confirm that number of relationships matches the number of clubs
print(f"The number of relationships match the number of clubs: {len(rel_located_in) == len(club_node)}")

rel_located_in.to_csv('./data/rel_located_in.csv', index=False)

The number of relationships match the number of clubs: True


## Load data into Neo4j

Follow the instructions from *step 3 onwards* in [Week 9's Lab](https://csse-uwa.gitbook.io/data-warehousing-lab-sheets/week-9-import-data-from-a-relational-database-into-neo4j#id-2.-design-and-etl) to import the .csv files into Neo4j.

Also, follow the instructions in [Week 11's Lab](https://csse-uwa.gitbook.io/data-warehousing-lab-sheets/week-11-advanced-cypher-and-apoc#a.-preparation) to install the APOC library which extends Cypher's functionality to allow for easier loading of data and more complex queries.  

### To delete any existing data in the database
```cypher
MATCH (n) DETACH DELETE n
```

### Query to load nodes  

```cypher
CALL apoc.import.csv(
    [
        {fileName: 'file:/club_node.csv', labels: ['Club']},
        {fileName: 'file:/country_node.csv', labels: ['Country']},
        {fileName: 'file:/player_node.csv', labels: ['Player']}
    ],
    [],
    {}
)
```

### Queries to convert datatypes of numerical and date variables

```
MATCH (p:Player)
SET p.player_id = toInteger(p.player_id)
SET p.age = toInteger(p.age)
SET p.height = toInteger(p.height)
SET p.dob = date({ year: toInteger(substring(p.dob, 6, 4)), month: toInteger(substring(p.dob, 3, 2)), day: toInteger(substring(p.dob, 0, 2))})

MATCH (c:Club)
SET c.club_id = toInteger(c.club_id)

MATCH (co:Country)
SET co.country_id = toInteger(co.country_id)
```

### Queries to load relationships

```cypher
LOAD CSV WITH HEADERS FROM 'file:///rel_plays_for.csv' AS row
MATCH (p:Player {player_id: toInteger(row.player_id)}), 
      (c:Club {club_id: toInteger(row.club_id)})
MERGE (p)-[:PLAYS_FOR]->(c)
```

```cypher
LOAD CSV WITH HEADERS FROM 'file:///rel_located_in.csv' AS row
MATCH (c:Club {club_id: toInteger(row.club_id)}),
    (y:Country {country_id: toInteger(row.club_country_id)})
MERGE (c)-[:LOCATED_IN]->(y)
```

```cypher
LOAD CSV WITH HEADERS FROM 'file:///rel_from.csv' AS row
MATCH (p:Player {player_id: toInteger(row.player_id)}),
    (c:Country {country_id: toInteger(row.country_id)})
MERGE (p)-[r:FROM]->(c)
SET r.caps = toInteger(row.caps)
SET r.intl_goals = toInteger(row.intl_goals)
SET r.jersey_num = toInteger(row.jersey_num)
```